In [16]:
import csv
import requests
import pandas as pd
from tqdm import tqdm
import pymongo
from pandas.io.json import json_normalize
import json

In [2]:
#Tip when dealing with zip codes. dtpye will default with integer, no bueno when it comes to zipcodes
#that start wth 0. SET the dtype to strig. 

zip_codes_file = pd.read_csv("../datasets/uszips.csv", dtype={'zip': 'str'})
zip_df = zip_codes_file
zip_df.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,00601,18.18004,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN,17242,111.4,72001,Adjuntas,"{'72001':99.43,'72141':0.57}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,00602,18.36073,-67.17517,Aguada,PR,Puerto Rico,True,NaN,38442,523.5,72003,Aguada,{'72003':100},Aguada,72003,False,False,America/Puerto_Rico
2,00603,18.45439,-67.12202,Aguadilla,PR,Puerto Rico,True,NaN,48814,667.9,72005,Aguadilla,{'72005':100},Aguadilla,72005,False,False,America/Puerto_Rico
3,00606,18.16724,-66.93828,Maricao,PR,Puerto Rico,True,NaN,6437,60.4,72093,Maricao,"{'72093':94.88,'72121':1.35,'72153':3.78}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,00610,18.29032,-67.12243,Anasco,PR,Puerto Rico,True,NaN,27073,312.0,72011,Añasco,"{'72003':0.55,'72011':99.45}",Añasco|Aguada,72011|72003,False,False,America/Puerto_Rico


In [3]:
zip_code = zip_df['zip'].tolist()

In [4]:
nj_zips_df = zip_df.groupby(['state_id']).get_group('NJ')
nj_zips_df.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
1963,07001,40.58338,-74.27004,Avenel,NJ,New Jersey,True,NaN,18307,1671.6,34023,Middlesex,{'34023':100},Middlesex,34023,False,False,America/New_York
1964,07002,40.66464,-74.10868,Bayonne,NJ,New Jersey,True,NaN,65300,4161.2,34017,Hudson,{'34017':100},Hudson,34017,False,False,America/New_York
1965,07003,40.81002,-74.18679,Bloomfield,NJ,New Jersey,True,NaN,48810,3437.6,34013,Essex,{'34013':100},Essex,34013,False,False,America/New_York
1966,07004,40.88278,-74.30415,Fairfield,NJ,New Jersey,True,NaN,7331,283.5,34013,Essex,{'34013':100},Essex,34013,False,False,America/New_York
1967,07005,40.92947,-74.42099,Boonton,NJ,New Jersey,True,NaN,15095,313.7,34027,Morris,{'34027':100},Morris,34027,False,False,America/New_York


In [5]:
#new_jersey_zip_codes =  nj_zips_df['zip'].tolist()
new_jersey_zip_codes = ["07307"]

In [6]:
url = "https://www.walmart.com/store/finder/electrode/api/stores?\
singleLineAddr=%s&serviceTypes=pharmacy&distance=1"
walmart_store_database = []

In [10]:
for zips in tqdm(new_jersey_zip_codes):
    store_response = requests.get(url % zips).json()
    store_data = store_response.get('payload',{}).get('storesData',{}).get("stores",[])
    for store in store_data:
        store_id = store.get('id')
        display_name = store.get("displayName")
        address = store.get("address")
        ziptest = store.get("address"[0])
        phone = store.get("phone")
        data = {"name":display_name,
                "store_id":store_id,
                "address":address,
                "phone":phone,
            
               }
        walmart_store_database.append(data)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.65it/s]


In [14]:
walmart_store_database

[{'name': 'Secaucus Supercenter',
  'store_id': 3520,
  'address': {'postalCode': '07094',
   'address': '400 Park Pl',
   'city': 'Secaucus',
   'state': 'NJ',
   'country': 'US'},
  'phone': '201-325-9280'},
 {'name': 'Kearny Supercenter',
  'store_id': 5447,
  'address': {'postalCode': '07032',
   'address': '150 Harrison Ave',
   'city': 'Kearny',
   'state': 'NJ',
   'country': 'US'},
  'phone': '201-955-0280'},
 {'name': 'North Bergen Supercenter',
  'store_id': 3795,
  'address': {'postalCode': '07047',
   'address': '2100 88th St',
   'city': 'North Bergen',
   'state': 'NJ',
   'country': 'US'},
  'phone': '201-758-2810'},
 {'name': 'Bayonne Supercenter',
  'store_id': 5867,
  'address': {'postalCode': '07002',
   'address': '500 Bayonne Crossing Way',
   'city': 'Bayonne',
   'state': 'NJ',
   'country': 'US'},
  'phone': '201-620-6137'},
 {'name': 'Teterboro Supercenter',
  'store_id': 3159,
  'address': {'postalCode': '07608',
   'address': '1 Teterboro Landing Drive',
   '

In [15]:
df = pd.DataFrame(walmart_store_database)
df.address.apply(pd.Series)

,postalCode,address,city,state,country
0,07094,400 Park Pl,Secaucus,NJ,US
1,07032,150 Harrison Ave,Kearny,NJ,US
2,07047,2100 88th St,North Bergen,NJ,US
3,07002,500 Bayonne Crossing Way,Bayonne,NJ,US
4,07608,1 Teterboro Landing Drive,Teterboro,NJ,US
...,...,...,...,...,...
93,11749,1850 Veterans Hwy,Islandia,NY,US
94,08809,Route 513 And I-78,Clinton,NJ,US
95,08724,1872 Route 88,Brick,NJ,US
96,08691,700 Marketplace Blvd,Hamilton,NJ,US


In [214]:
with open('new_jersey_test.json', 'w') as outfile:
    json.dump(walmart_store_database, outfile)

In [218]:
# with open('new_jersey_test.json') as json_file:
#     data = json.load(json_file)

pd.read_json(r'new_jersey_test.json')

,name,store_id,address,phone
0,Linden Supercenter,3469,"{'postalCode': '07036', 'address': '1050 W Edg...",908-474-9055
1,Woodbridge Store,5281,"{'postalCode': '07095', 'address': '360 Us Hig...",732-826-4652
2,Edison Store,5201,"{'postalCode': '08817', 'address': '2220 State...",732-650-1297
3,Union Store,3292,"{'postalCode': '07083', 'address': '900 Spring...",908-624-0644
4,Watchung Store,5111,"{'postalCode': '07069', 'address': '1501 Us Hi...",908-756-1925
...,...,...,...,...
27242,Philadelphia Store,5130,"{'postalCode': '19114', 'address': '9745 Roose...",215-698-0350
27243,Franklin Store,2090,"{'postalCode': '07416', 'address': '230 State ...",973-209-4242
27244,Willow Grove Supercenter,3564,"{'postalCode': '19090', 'address': '2101 Blair...",215-830-8370
27245,Cinnaminson Supercenter,2871,"{'postalCode': '08077', 'address': '2501 Route...",856-303-2119


In [181]:
conn = 'mongodb://localhost:27017'
# Pass connection to the pymongo instance.
client = pymongo.MongoClient(conn)
# Connect to a database. Will create one if not already available.
db = client.walmart_stores_db
db.stores.drop()

In [182]:
stores = db.stores
stores.insert_many(walmart_store_database)

TypeError: 'Collection' object is not callable. If you meant to call the 'head' method on a 'Collection' object it is failing because no such method exists.